In [ ]:
#Ejemplo de cómo implementar una red neuronal para
#clasificación con (MLPClassifier) utilizando Sklearn

from sklearn.neural_network import MLPClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

#cargar el conjunto de datos Iris
iris = load_iris()
X, y = iris.data, iris.target

#Explorando los datos (opcional)
type(iris)
iris.keys()
iris['data']
iris['target']
iris['target_names']
iris['DESCR']
iris['feature_names']


['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [ ]:
# Dividir los datos en conjuntos de entramiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar los datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Crear una instanci de MLPClassifier
mlp_clf = MLPClassifier(hidden_layer_sizes=(100),
                        activation='relu',
                        solver='adam',
                        max_iter=100,
                        random_state=42)

# Entrenar el modelo
mlp_clf.fit(X_train_scaled, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=100, max_iter=100, random_state=42)

In [ ]:
# Realizamos predicciones en el conjunto de prueba

Y_pred = mlp_clf.predict(X_test_scaled)
print(y_test)
print(Y_pred)


[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]
[1 0 2 1 2 0 1 2 1 1 2 0 0 0 0 2 2 1 1 2 0 2 0 2 2 2 2 2 0 0]


In [ ]:
# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, Y_pred)
print("Precisión del modelo:", accuracy)

Precisión del modelo: 0.9333333333333333


In [ ]:
#Ejemplo básico de cómo implementar una red
#neuronal para regresión utilizando MLPRegressor de
#Scikit-learn

from sklearn.neural_network import MLPRegressor
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# Cargar el conjunto de datos de California Housing
housing = fetch_california_housing()
X,y = housing.data, housing.target

# Explorando los datos (Opcional)
type(housing)
housing.keys()
housing['data']
housing['target']
housing['target_names']
housing['DESCR']
housing['feature_names']

['MedInc',
 'HouseAge',
 'AveRooms',
 'AveBedrms',
 'Population',
 'AveOccup',
 'Latitude',
 'Longitude']

In [ ]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar las características para un mejor rendimiento del modelo
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Crear una instancia de MLPRegressor
mlp_reg = MLPRegressor(hidden_layer_sizes=(100),
                       activation='relu',
                       solver='adam',
                       max_iter=100,
                       random_state=42)
# Entrenar el modelo
mlp_reg.fit(X_train_scaled, y_train)

# Realizar predicciones en el cojunto de prueba
y_pred = mlp_reg.predict(X_test_scaled)

# Calcular el error cuadrático medio (MSE)
mse = mean_squared_error(y_test, y_pred)
print("Error cuadrático medio (MSE):", mse)


Error cuadrático medio (MSE): 0.32870115322611493


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, RMSprop, SGD

X,Y = housing.data, housing.target

scaler = MinMaxScaler()
X_scale = scaler.fit_transform(X)
Y_scale = scaler.fit_transform(Y.reshape(-1, 1)).flatten()  # Flatten para asegurar una dimensión correcta

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, Y_train, Y_test = train_test_split(X_scale, Y_scale, test_size=0.2, random_state=42)

# Definir una función para crear el modelo
def create_model(optimizer='adam', hidden_layer_sizes=(32, 32), activation='relu'):
    model = Sequential()
    model.add(Dense(hidden_layer_sizes[0], activation=activation, input_shape=(X_train.shape[1],)))
    for units in hidden_layer_sizes[1:]:
        model.add(Dense(units, activation=activation))
    model.add(Dense(1))  # Capa de salida para regresión lineal
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['accuracy'])
    return model

# Función para evaluar el modelo con diferentes configuraciones
def evaluate_model(X_train, Y_train, X_test, Y_test, optimizer, hidden_layer_sizes, activation, epochs):
    model = create_model(optimizer=optimizer, hidden_layer_sizes=hidden_layer_sizes, activation=activation)
    model.fit(X_train, Y_train, epochs=epochs, verbose=0)
    Y_pred = model.predict(X_test).flatten()

    # Calcular métricas de evaluación
    accuracy = model.evaluate(X_test, Y_test)[1]
    r2 = r2_score(Y_test, Y_pred)
    mse = mean_squared_error(Y_test, Y_pred)

    return optimizer, hidden_layer_sizes, activation, epochs, accuracy, r2, mse

# Definir configuraciones a probar
optimizers = ['adam', 'rmsprop', 'sgd']
hidden_layer_sizes = [(32,), (64,), (32, 32), (64, 32)]
activations = ['relu', 'sigmoid']
epochs = [20, 30, 40]

# Lista para almacenar resultados
results = []

# Iterar sobre todas las combinaciones de parámetros
for opt in optimizers:
    for hl_sizes in hidden_layer_sizes:
        for act in activations:
            for ep in epochs:
                result = evaluate_model(X_train, Y_train, X_test, Y_test, opt, hl_sizes, act, ep)
                results.append(result)

# Crear un DataFrame para visualizar los resultados
results_df = pd.DataFrame(results, columns=['Optimizer', 'Hidden Layer Sizes', 'Activation', 'Epochs', 'Accuracy', 'R2 Score', 'MSE'])
results_df.sort_values(by='MSE', ascending=True, inplace=True)  # Ordenar por MSE ascendente o la métrica que prefieras

# Mostrar tabla resumen
print("Tabla Resumen de Resultados:")
print(results_df.head(10))  # Mostrar los 10 mejores resultados

# Guardar los resultados en un archivo CSV si deseas
results_df.to_csv('results_summary.csv', index=False)

129/129 [==============================] - 0s 1ms/step - loss: 0.0331 - accuracy: 0.0274
Tabla Resumen de Resultados:
   Optimizer Hidden Layer Sizes Activation  Epochs  Accuracy  R2 Score  \
20      adam           (64, 32)       relu      40  0.039729  0.748432   
14      adam           (32, 32)       relu      40  0.039729  0.727665   
19      adam           (64, 32)       relu      30  0.038033  0.716795   
44   rmsprop           (64, 32)       relu      40  0.040213  0.703946   
8       adam              (64,)       relu      40  0.038517  0.702017   
12      adam           (32, 32)       relu      20  0.038275  0.701181   
18      adam           (64, 32)       relu      20  0.037791  0.698223   
38   rmsprop           (32, 32)       relu      40  0.037548  0.691124   
13      adam           (32, 32)       relu      30  0.036822  0.687871   
7       adam              (64,)       relu      30  0.039002  0.680156   

         MSE  
20  0.014014  
14  0.015171  
19  0.015777  
44  0.0